In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Load and Print Data

In [ ]:
def load_and_print_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[:5]:
            print(line.strip())

    return lines[:5]

data = load_and_print_data('/content/drive/MyDrive/Colab Notebooks/NLP/NLP project/twitter.txt')

Split to Sentences

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def load_and_print_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return ''.join(lines)  # Join all lines to form a single string

def split_to_sentences(data):
    sentences = sent_tokenize(data)
    return sentences

# Example usage
file_path = '/content/drive/MyDrive/Colab Notebooks/NLP/NLP project/twitter.txt'
data = load_and_print_data(file_path)
sentences = split_to_sentences(data)
print(sentences [:5])


Tokenize Sentences

In [ ]:
def tokenize_sentences(sentences):
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    return tokenized_sentences

# Example usage
tokenized_sentences = tokenize_sentences(sentences)
print (tokenized_sentences [:5])

Tokenize Data

In [ ]:
def tokenize_data(data):
    tokens = nltk.word_tokenize(data)
    return tokens

tokens = tokenize_data(data)
print (tokens [:5])


['How', 'are', 'you', '?', 'Btw']


Test / Train Split

In [ ]:
from sklearn.model_selection import train_test_split

def test_train_split(data, test_size=0.2):
    train_data, test_data = train_test_split(data, test_size=test_size)
    return train_data, test_data

train_data, test_data = test_train_split(sentences)

Count Words

In [ ]:
from collections import Counter

def count_words(tokenized_sentences):
    words = [word for sentence in tokenized_sentences for word in sentence]
    word_counts = Counter(words)
    return word_counts

# Example usage
word_counts = count_words(tokenized_sentences)
print (word_counts)


Counter({'.': 29626, '!': 26058, 'the': 17116, 'I': 16153, 'to': 15451, ',': 15289, 'a': 11672, 'you': 10641, '?': 8502, 'and': 8152, ':': 7727, 'for': 7534, 'is': 7272, 'in': 7269, 'of': 7118, 'it': 6401, "'s": 5994, '#': 5389, 'on': 5321, 'my': 5003, 'that': 4830, "n't": 4187, ')': 4076, '...': 4063, 'me': 3971, 'be': 3674, 'at': 3608, 'with': 3384, 'have': 3231, 'are': 3123, 'your': 3123, "''": 3052, 'do': 3013, 'this': 2766, "'m": 2594, 'so': 2571, '``': 2507, 'like': 2386, 'was': 2354, 'i': 2351, 'just': 2261, 'out': 2237, 'not': 2222, 'up': 2185, '&': 2160, 'all': 2144, 'but': 2107, 'we': 2076, 'get': 2011, '(': 2003, '-': 1899, 'can': 1788, 'RT': 1770, 'will': 1762, 'about': 1759, 'The': 1708, 'what': 1656, 'love': 1631, 'from': 1614, 'know': 1553, 'good': 1532, 'one': 1476, 'day': 1460, 'now': 1459, 'You': 1416, 'if': 1396, '..': 1358, 'It': 1355, 'u': 1341, 'time': 1340, 'an': 1336, 'today': 1314, 'they': 1313, 'when': 1305, 'or': 1303, 'as': 1261, 'We': 1244, "'re": 1241, 'go

Words with N
Frequency

In [ ]:
def words_with_n_frequency(tokenized_sentences, threshold_value):
    word_counts = count_words(tokenized_sentences)
    frequent_words = {word: count for word, count in word_counts.items() if count >= threshold_value}
    return frequent_words

frequent_words = words_with_n_frequency(tokenized_sentences, threshold_value=2)


Replace OOV with UNK

In [ ]:
def replace_oov_with_unk(tokenized_sentences, vocabulary, unknown_token='<UNK>'):
    processed_sentences = [
        [word if word in vocabulary else unknown_token for word in sentence]
        for sentence in tokenized_sentences
    ]
    return processed_sentences

vocabulary = set(frequent_words.keys())
processed_sentences = replace_oov_with_unk(tokenized_sentences, vocabulary)


Pre_Process Data

In [ ]:
def pre_process_data(train_data, test_data, threshold_value):
    train_tokenized = tokenize_sentences(train_data)
    test_tokenized = tokenize_sentences(test_data)

    frequent_words = words_with_n_frequency(train_tokenized, threshold_value)
    vocabulary = set(frequent_words.keys())
    vocabulary.update(["<s>", "</s>"])
    train_processed = replace_oov_with_unk(train_tokenized, vocabulary)
    test_processed = replace_oov_with_unk(test_tokenized, vocabulary)

    return train_processed, test_processed

train_data, test_data = test_train_split(sentences)

train_processed, test_processed = pre_process_data(train_data, test_data, threshold_value=25)

Count N-Grams

In [ ]:
from collections import defaultdict
def count_ngrams(data, n, start_token='<s>', end_token='</s>'):
    ngrams = defaultdict(int)
    for sentence in data:
        padded_sentence = [start_token] * (n-1) + sentence + [end_token]
        for i in range(len(padded_sentence) - n + 1):
            ngram = tuple(padded_sentence[i:i+n])
            ngrams[ngram] += 1
    return ngrams

n = 2
train_bigrams = count_ngrams(train_processed, n)
train_trigrams = count_ngrams(train_processed, n + 1)
test_trigrams = count_ngrams(test_processed, n)
test_bigrams = count_ngrams(test_processed, n)

print("Train Bigrams:", train_bigrams)
print("Test Bigrams:", test_bigrams)

Train Bigrams: defaultdict(<class 'int'>, {('<s>', 'Ok'): 30, ('Ok', '#'): 1, ('#', '<UNK>'): 3732, ('<UNK>', 'was'): 316, ('was', '<UNK>'): 375, ('<UNK>', ','): 5085, (',', 'got'): 12, ('got', 'it'): 39, ('it', '.'): 446, ('.', '</s>'): 23149, ('<s>', 'Then'): 56, ('Then', 'move'): 1, ('move', 'along'): 1, ('along', 'as'): 1, ('as', 'they'): 8, ('they', 'try'): 1, ('try', 'to'): 60, ('to', '<UNK>'): 2328, ('<UNK>', 'it'): 637, ('it', 'out'): 106, ('out', '.'): 113, ('<s>', '#'): 913, ('<UNK>', 'each'): 19, ('each', 'other'): 28, ('other', 'Me'): 1, ('Me', ':'): 32, (':', 'did'): 3, ('did', 'you'): 90, ('you', 'know'): 174, ('know', 'that'): 65, ('that', 'a'): 32, ('a', '<UNK>'): 3524, ('<UNK>', 'has'): 288, ('has', 'two'): 3, ('two', '<UNK>'): 63, ('<UNK>', 'on'): 1398, ('on', 'its'): 7, ('its', '<UNK>'): 102, ('<UNK>', '?'): 2096, ('?', '</s>'): 5647, ('<s>', 'thank'): 80, ('thank', 'you'): 194, ('you', '!'): 358, ('!', '!'): 4841, ('!', '</s>'): 15382, ('<s>', 'Nice'): 53, ('Nice', 

Estimate Probability of
Single Word

In [ ]:
def estimate_probability_single_word(word, previous_ngram, ngram_counts, n_plus_1_gram_counts, vocabulary_size, k=1):
    previous_ngram = tuple(previous_ngram)
    ngram = previous_ngram + (word,)

    ngram_count = n_plus_1_gram_counts.get(ngram, 0)
    previous_ngram_count = ngram_counts.get(previous_ngram, 0)

    probability = (ngram_count + k) / (previous_ngram_count + k * vocabulary_size)
    return probability
vocabulary = set(word for sentence in train_processed for word in sentence)
vocabulary_size = len(vocabulary)
previous_ngram = ["this"]
word_prob = estimate_probability_single_word("is", previous_ngram, train_bigrams, train_bigrams, vocabulary_size)
print("Probability of 'is' given ['this']:", word_prob)

Probability of 'is' given ['this']: 0.06880733944954129


Estimate Probability of
All Words

In [ ]:
def estimate_probability_all_words(previous_ngram, ngram_counts, n_plus_1_gram_counts, vocabulary, k=1):
    probabilities = {}
    vocabulary_size = len(vocabulary)
    for word in vocabulary:
        probabilities[word] = estimate_probability_single_word(word, previous_ngram, ngram_counts, n_plus_1_gram_counts, vocabulary_size, k)
    return probabilities
all_word_probs = estimate_probability_all_words(previous_ngram, train_bigrams, train_bigrams, vocabulary)
print("Probabilities of all words given ['this']:", all_word_probs)


Probabilities of all words given ['this']: {'forgot': 0.0005096839959225281, 'gone': 0.0005096839959225281, 'worry': 0.0005096839959225281, 'swear': 0.0005096839959225281, 'headed': 0.0005096839959225281, 'faith': 0.0005096839959225281, 'Mom': 0.0005096839959225281, 'America': 0.0005096839959225281, 'his': 0.0005096839959225281, 'mom': 0.0010193679918450561, 'blog': 0.0005096839959225281, 'minutes': 0.0005096839959225281, 'some': 0.0010193679918450561, 'oh': 0.0005096839959225281, 'plan': 0.0005096839959225281, 'cards': 0.0005096839959225281, "'S": 0.0005096839959225281, 'Did': 0.0005096839959225281, 'celebrate': 0.0005096839959225281, 'Ready': 0.0005096839959225281, 'problems': 0.0005096839959225281, 'stopped': 0.0005096839959225281, 'tomorrow': 0.0005096839959225281, 'studio': 0.0005096839959225281, 'somebody': 0.0005096839959225281, 'feed': 0.0005096839959225281, 'Facebook': 0.0005096839959225281, 'easier': 0.0005096839959225281, 'let': 0.0005096839959225281, 'hahah': 0.000509683995

Make Count Matrix

In [ ]:
# Function to make a count matrix
def make_count_matrix(n_plus_1_gram_counts, vocabulary):
    vocab_list = list(vocabulary)
    vocab_size = len(vocab_list)
    count_matrix = np.zeros((vocab_size, vocab_size))

    # Inside the make_count_matrix function
    print("Vocabulary List:", vocab_list)
    print("Vocabulary Size:", vocab_size)

    for ngram, count in n_plus_1_gram_counts.items():
        if len(ngram) == 2:  # Assuming bigrams
            try:
                i = vocab_list.index(ngram[0])
                j = vocab_list.index(ngram[1])
                count_matrix[i, j] = count
                print(f"Adding count {count} to matrix position ({i}, {j}) for ngram {ngram}")
            except ValueError as e:
                print(f"Error: {e} - Ngram: {ngram}")
      # Inside the make_count_matrix function, after the for loop
    for ngram in train_bigrams.keys():
      if '<s>' in ngram:
        print(ngram)
    return count_matrix

count_matrix = make_count_matrix(train_bigrams, vocabulary)
print("Count Matrix:\n", count_matrix)
print(f"Total sum of counts in the count matrix: {np.sum(count_matrix)}")

Streaming output truncated to the last 5000 lines.
Adding count 1 to matrix position (1736, 69) for ngram ('in', 'sales')
Adding count 1 to matrix position (1759, 49) for ngram ('your', 'glad')
Adding count 1 to matrix position (99, 334) for ngram ("''", 'Boston')
Adding count 1 to matrix position (546, 1185) for ngram ('Is', 'hilarious')
Adding count 1 to matrix position (1751, 439) for ngram ('be', 'yourself')
Adding count 1 to matrix position (1824, 313) for ngram ('say', 'Thanks')
Adding count 1 to matrix position (838, 816) for ngram ('God', 'sometimes')
Adding count 1 to matrix position (816, 386) for ngram ('sometimes', 'comes')
Adding count 1 to matrix position (748, 259) for ngram ('things', 'become')
Adding count 1 to matrix position (259, 1237) for ngram ('become', 'clear')
Adding count 1 to matrix position (837, 400) for ngram ('even', 'fashion')
Adding count 1 to matrix position (629, 1343) for ngram ('seeing', 'Chicago')
Adding count 1 to matrix position (703, 911) for ng

Make Probability
Matrix

In [ ]:
def make_probability_matrix(n_plus_1_gram_counts, vocabulary, k=1):
    vocab_list = list(vocabulary)
    vocab_size = len(vocab_list)
    count_matrix = np.zeros((vocab_size, vocab_size))

    for ngram, count in n_plus_1_gram_counts.items():
        if len(ngram) == 2:  # Assuming bigrams
            try:
                i = vocab_list.index(ngram[0])
                j = vocab_list.index(ngram[1])
                count_matrix[i, j] = count
            except ValueError as e:
                print(f"Error: {e} - Ngram: {ngram}")

    # Apply smoothing by adding k to each count
    smoothed_count_matrix = count_matrix + k
    row_sums = smoothed_count_matrix.sum(axis=1, keepdims=True)

    # Calculate probability matrix
    with np.errstate(divide='ignore', invalid='ignore'):
        probability_matrix = np.divide(smoothed_count_matrix, row_sums, where=row_sums != 0)

    return probability_matrix

probability_matrix = make_probability_matrix(train_trigrams, vocabulary, k=1)
print("Probability Matrix:\n", probability_matrix)


Probability Matrix:
 [[0.00050968 0.00050968 0.00050968 ... 0.00050968 0.00050968 0.00050968]
 [0.00050968 0.00050968 0.00050968 ... 0.00050968 0.00050968 0.00050968]
 [0.00050968 0.00050968 0.00050968 ... 0.00050968 0.00050968 0.00050968]
 ...
 [0.00050968 0.00050968 0.00050968 ... 0.00050968 0.00050968 0.00050968]
 [0.00050968 0.00050968 0.00050968 ... 0.00050968 0.00050968 0.00050968]
 [0.00050968 0.00050968 0.00050968 ... 0.00050968 0.00050968 0.00050968]]



Calculate Perplexity

In [ ]:
import numpy as np
def calculate_perplexity(sentence, ngram_counts, n_plus_1_gram_counts, vocabulary_size, k=1):
    n = len(next(iter(ngram_counts)))  # Length of n-gram
    sentence = ["<s>"] * (n-1) + sentence + ["</s>"]
    N = len(sentence)
    log_prob = 0.0

    for i in range(n-1, N):
        previous_ngram = sentence[i-n+1:i]
        word = sentence[i]
        probability = estimate_probability_single_word(word, previous_ngram, ngram_counts, n_plus_1_gram_counts, vocabulary_size, k)
        log_prob += np.log(probability)

    perplexity = np.exp(-log_prob / N)
    return perplexity
sample_sentence = ["I", "love", "you"]
perplexity = calculate_perplexity(sample_sentence, train_bigrams, train_bigrams, vocabulary_size, k=1)
print("Perplexity:", perplexity)

Perplexity: 8.378811324525786


Suggest a Word

In [ ]:
def suggest_a_word(previous_tokens, ngram_counts, n_plus_1_gram_counts, vocabulary_size, k=1, start_with=None):
    previous_ngram = tuple(previous_tokens)

    probabilities = estimate_probability_all_words(
        previous_ngram, ngram_counts, n_plus_1_gram_counts, vocabulary, k
    )

    # Filter probabilities if start_with is provided
    if start_with:
        probabilities = {word: prob for word, prob in probabilities.items() if word.startswith(start_with)}

    if not probabilities:
        return None

    # Suggest the word with the highest probability
    suggested_word = max(probabilities, key=probabilities.get)

    return suggested_word

def estimate_probability_all_words(previous_ngram, ngram_counts, n_plus_1_gram_counts, vocabulary, k=1):
    probabilities = {}
    vocabulary_size = len(vocabulary)
    for word in vocabulary:
        probabilities[word] = estimate_probability_single_word(word, previous_ngram, ngram_counts, n_plus_1_gram_counts, vocabulary_size, k)
    return probabilities
previous_tokens = ["I", "love"]
suggested_word = suggest_a_word(previous_tokens, train_bigrams, train_trigrams, vocabulary_size, k=1)
print("Suggested Word:", suggested_word)

Suggested Word: you


Get Suggestions

In [ ]:
def get_suggestions(previous_tokens, ngram_counts, n_plus_1_gram_counts, vocabulary, k=1, start_with=None, num_suggestions=5):
    previous_ngram = tuple(previous_tokens)

    probabilities = estimate_probability_all_words(
        previous_ngram, ngram_counts, n_plus_1_gram_counts, vocabulary, k
    )

    # Filter probabilities if start_with is provided
    if start_with:
        probabilities = {word: prob for word, prob in probabilities.items() if word.startswith(start_with)}

    if not probabilities:
        return []

    # Sort words based on probabilities and get the top N suggestions
    sorted_words = sorted(probabilities.items(), key=lambda item: item[1], reverse=True)
    suggestions = [word for word, prob in sorted_words[:num_suggestions]]

    return suggestions

# Example usage
previous_tokens = ["love", "is"]
suggestions = get_suggestions(previous_tokens, train_bigrams, train_trigrams, vocabulary, k=1, num_suggestions=5)
print("Suggestions:", suggestions)


Suggestions: ['much', 'not', 'one', 'like', 'in']
